In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

# The function used to generate data, the Runge function
def Runge_func(x):
    return 1.0/(1 + 25*x**2)

In [2]:
# Bias-variance-tradeoff, using bootstrap and OLS

# Setting up dataset
np.random.seed(2025)
n = 1000
x = np.linspace(-1, 1, n)
y = Runge_func(x) + np.random.normal(0, 0.1, size=n)

x = x.reshape(-1,1)
y = y.reshape(-1,1)

bootstraps = 10

max_degree = 35
poly_degrees = np.arange(1, max_degree + 1, 1)

biases = np.zeros(max_degree)
variances = np.zeros(max_degree)
MSEs = np.zeros(max_degree)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

for degree in range(len(poly_degrees)):
    np.random.seed(2025 + degree)
    X_train = polynomial_features(x_train, poly_degrees[degree], intercept=True)
    X_test = polynomial_features(x_test, poly_degrees[degree], intercept=True)

    # Scaling of the data
    scaler = StandardScaler()   # initialize scaler method
    scaler.fit(X_train)               # only base the scaling on X_train, to prevent data leakage from X_test 
    X_train_s = scaler.transform(X_train)
    X_test_s = scaler.transform(X_test)
    
    # predictions are later filled with predictions made from X_test (feature matrix), constructed from x_test, 
    # so got to have same length
    predictions = np.zeros([bootstraps, len(x_test)])

    for b in range(bootstraps):
        np.random.seed(1025 + b)
        # For each bootstrap sample of X_train and Y_train, we train model, predict on X_test
        # Later comparing against the un-touched y_test
        X_train_resampled, y_train_resampled = resample(X_train_s, y_train)

        # Training the model on training data
        beta = OLS_parameters(X_train_resampled, y_train_resampled)
        # Making the prediction on test data
        pred = X_test_s @ beta
        predictions[b,:] = pred.ravel()

    # We take the true values or target, as the un-tough values in the y_test split
    # The predicted y values, lives in the predictions matrix, where each row is a sample of values,
    # and each column corresponding to a one y point across bootstrap samples 
    biases[degree] = np.mean((y_test.T - np.mean(predictions, axis=0))**2)

    # Var(prediction) is the mean of the flatend matrix, over all samples
    variances[degree] = np.mean((predictions - np.mean(predictions, axis=0))**2)

    # For the MSE, we take difference of each y point per bootstrap sample, making y_test a row vector
    # then squaring, before taking the mean over the flattened matrix
    
    MSEs[degree] = np.mean(np.mean((predictions - y_test.T)**2, axis=1), axis=0)

plt.plot(poly_degrees, biases, 'o-', label="Bias")
plt.plot(poly_degrees, variances, 'o-', label="Variance")
plt.plot(poly_degrees, MSEs, 'o-', label="MSE")
#plt.title(f"Bias-variance tradeoff, for {n} data points and {bootstraps} bootstraps")
plt.xlabel("Polynomial degree")
plt.ylabel("Error")
plt.title("OLS")
plt.legend()
plt.savefig("bias_var_tradeoff.pdf", bbox_inches="tight")
plt.show()
plt.close()

NameError: name 'polynomial_features' is not defined